In [15]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.io import read_image
import os
import PIL
from PIL import Image
from torchvision.transforms import v2
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler


In [16]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform

        self.images_files = sorted(os.listdir(images_dir))
        self.labels_files = sorted(os.listdir(labels_dir))

    def __len__(self):
        return len(self.images_files)
    
    def __getitem__(self, k):
        
        image_name = self.images_files[k]
        #print(self.images_files[k])
        image_path = os.path.join(self.images_dir, image_name)
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        #print(self.labels_files[k])
        #print(self.labels_files[k])
        label_name = self.labels_files[k]
        label_path = os.path.join(self.labels_dir, label_name)
        with open(label_path, 'r') as file:
            #split the label
            full_label = file.readline().split(" ")
            label = int(full_label[0]) 
            bbox = (float(full_label[1]), float(full_label[2]), float(full_label[3]), float(full_label[4]))

        return image, label, torch.tensor(bbox, dtype=torch.float32)
        #return image, torch.tensor(label, dtype=torch.long), torch.tensor(bbox, dtype=torch.float32)


In [17]:
#print(len(os.listdir("crop_dataset/images"))) = 53157 (without background images)
#dataset contains 1662 batch, each one of size 32
#Total = 1662*32 = 53184
data = CustomDataset("real_images/images", "real_images/labels", transform=transform)
#train data and val data split
train_size = int(len(data) * 0.7)
val_size = len(data)- train_size
train_set, val_set = torch.utils.data.random_split(data, [train_size, val_size])

#Must be smaller for real images
batch_size = 4
train_data = DataLoader(train_set, batch_size=batch_size)
val_data = DataLoader(val_set, batch_size=batch_size)

# Iterate over the DataLoader
for batch in train_data:
    images, labels, bbox = batch
    print("Batch of images shape:", images.shape)
    print("Batch of labels:", labels.shape)
    print("BBOX:", bbox.shape)
    break

IndexError: list index out of range

In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [ ]:
# Define model
class SimpleNet(nn.Module):
    # def __init__(self):
    #     super().__init__()
    #     self.flatten = nn.Flatten()
    #     self.linear_relu_stack = nn.Sequential(
    #         nn.Linear(48*48*3, 128),
    #         nn.ReLU(),
    #         nn.Linear(128, 12),
    #     )
    
    def __init__(self, num_classes):
        super(SimpleNet, self).__init__()
        self.flatten = nn.Flatten()
        self.mod1 = nn.Sequential(
            nn.Conv2d(3, 3, (2,2), 1),
            nn.ReLU(),
            nn.Conv2d(3, 3, (2,2), 1),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc_label = nn.Linear(147852, num_classes)
        self.fc_bbox = nn.Linear(147852, 4)

    def forward(self, image):
        x = self.mod1(image)
        label = self.fc_label(x)
        bbox = self.fc_bbox(x)
        #x = nn.Flatten()
        return label, bbox

num_classes = 13
model = SimpleNet(num_classes).to(device)
print(model)

SimpleNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (mod1): Sequential(
    (0): Conv2d(3, 3, kernel_size=(2, 2), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(3, 3, kernel_size=(2, 2), stride=(1, 1))
    (3): ReLU()
    (4): Flatten(start_dim=1, end_dim=-1)
  )
  (fc_label): Linear(in_features=147852, out_features=13, bias=True)
  (fc_bbox): Linear(in_features=147852, out_features=4, bias=True)
)


In [ ]:
#separate loss for label and bbox
label_loss = nn.CrossEntropyLoss()
bbox_loss = nn.SmoothL1Loss()

learnin_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learnin_rate)

In [ ]:
def train(data, model, label_loss, bbox_loss, optimizer):
    model.train()
    size = len(data.dataset)
    batch_count = 1
    correct = 0
    loss_sum = 0
    print(enumerate(data))
    #for batch, block in enumerate(data):
    for batch, block in enumerate(data):
        print("dfasjfjas")
        images, labels, bbox = block[0].to(device), block[1].to(device), block[2].to(device)

        # Compute prediction error
        pred_labels, pred_bbox = model(images)
        loss_1 = label_loss(pred_labels, labels)
        loss_2 = bbox_loss(pred_bbox, bbox)

        #loss = label_loss.item() + bbox_loss.item()
        loss_sum += loss_1.item() + loss_2.item()

        # Backpropagation
        (loss_1 + loss_2).backward()
        optimizer.step()
        optimizer.zero_grad()

        correct += (pred_labels.argmax(1) == labels).type(torch.float).sum().item()

        batch_count += 1
    loss_sum /= batch_count
    correct /= size
    print(f"Training Error: \n Accuracy: {(100*correct):>0.1f}%, Average Loss: {loss_sum:>8f}")

In [ ]:
def test(data, model, loss_fn):
    size = len(data.dataset)
    num_batches = len(data)
    model.eval()
    test_loss, correct = 0, 0
    batch_count = 0
    with torch.no_grad():
        for batch in data:
            images, labels = batch
            images, labels = images.to(device), labels.to(device)

            # Compute prediction error
            pred = model(images)
            test_loss += loss_fn(pred, labels).item()
            correct += (pred.argmax(1) == labels).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_data, model, label_loss,bbox_loss, optimizer)
    #test(val_data, model, label_loss,bbox_loss)
print("Done!")

Epoch 1
-------------------------------
000015.jpg
000019.txt
000017.jpg
000021.txt
000032.jpg
000036.txt
000018.jpg
000022.txt
dfasjfjas
000000.jpg
000000.txt
000041.jpg


IndexError: list index out of range